## Entraîner plusieurs modèles de classification 

**Entraîner plusieurs modèles de classification (RandomForest, LogisticRegression, SVC)**

1. Logger chaque run dans MLflow

2. Visualiser et comparer les performances

3. Voir toutes les choses utiles à explorer dans l’interface MLflow

In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from mlflow.models.signature import infer_signature

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Charger les données
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Liste de classifieurs à comparer
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, solver='liblinear', random_state=42),
    "SVC": SVC(probability=True, kernel="rbf", random_state=42)
}

# Dossier pour les images
os.makedirs("plots", exist_ok=True)

# Lancer les runs
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        cm = confusion_matrix(y_test, y_pred)

        # Log des paramètres et métriques
        mlflow.log_param("model_name", model_name)
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("f1_score", f1)

        # Signature du modèle
        signature = infer_signature(X_test, y_pred)
        mlflow.sklearn.log_model(model, "model", signature=signature, input_example=X_test.iloc[:1])

        # Matrice de confusion
        plt.figure(figsize=(5, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
        plt.title(f"Confusion Matrix - {model_name}")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        fig_path = f"plots/cm_{model_name}.png"
        plt.tight_layout()
        plt.savefig(fig_path)
        mlflow.log_artifact(fig_path)
        plt.close()


 Ce qui est possible d'explorer dans MLflow UI
1. Runs
Compare les différents modèles que tu as testés

Tri par accuracy, f1_score…

2. Parameters
Voir les paramètres loggés (model_name, n_estimators, kernel, etc.)

3. Metrics
Compare plusieurs modèles dans un graphe de performance

Ex : accuracy et f1_score dans une vue parallèle

4. Artifacts
Visualise les images loggées comme les matrices de confusion

Les fichiers .png apparaissent dans chaque run

5. Models (si activé)
Voir l’historique des modèles enregistrés

6. Possibilité de les servir via API ou de les télécharger
